In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import os


In [3]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import random_split, DataLoader
import timm  # For EfficientNet

# Define image transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset from RealWaste/ (without train/val folders)
dataset_path = "realwaste-main/RealWaste"
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Split dynamically into train (80%) and validation (20%)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Total images: {len(dataset)} | Train: {len(train_dataset)} | Val: {len(val_dataset)}")


C:\Users\Srirama V Swamy\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total images: 4752 | Train: 3801 | Val: 951


In [4]:
import torch.nn as nn
import torch.optim as optim

# Load pre-trained EfficientNet-B0
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=len(dataset.classes))

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss, correct = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

        train_acc = 100 * correct / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.2f}%")

train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)


Epoch 1/20 | Loss: 11.0081 | Train Accuracy: 97.03%
Epoch 2/20 | Loss: 11.8074 | Train Accuracy: 96.90%
Epoch 3/20 | Loss: 9.6971 | Train Accuracy: 97.50%
Epoch 4/20 | Loss: 9.6590 | Train Accuracy: 97.37%
Epoch 5/20 | Loss: 14.9422 | Train Accuracy: 96.47%
Epoch 6/20 | Loss: 13.4066 | Train Accuracy: 96.21%


In [6]:
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            correct += (outputs.argmax(1) == labels).sum().item()

    val_acc = 100 * correct / len(val_loader.dataset)
    print(f"Validation Accuracy: {val_acc:.2f}%")

evaluate_model(model, val_loader)


Validation Accuracy: 85.17%
